In [ ]:
#!{sys.executable} -m pip install --upgrade psycopg2-binary

In [1]:
# ket noi
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Kết nối với user postgres
conn = psycopg2.connect(
    host="localhost",
    user="postgres",       # user mặc định
    password="123",           # nếu có mật khẩu cho postgres thì điền vào
)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()


In [8]:
# Tạo database nếu chưa có
db_name = "video_db"
cursor.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{db_name}';")
exists = cursor.fetchone()
if not exists:
    cursor.execute(f"CREATE DATABASE {db_name};")
    print(f"Database '{db_name}' created successfully!")
else:
    print(f"Database '{db_name}' already exists.")

cursor.close()
conn.close()

Database 'video_db' created successfully!


In [2]:
# Kết nối tới database vừa tạo
conn = psycopg2.connect(
    host="localhost",
    database="video_db",
    user="postgres",
    password="123"   # nếu user khác thì điền mật khẩu
)
cursor = conn.cursor()


In [ ]:
create_tables_sql = """
CREATE TABLE IF NOT EXISTS videos (
    id SERIAL PRIMARY KEY,
    video_path VARCHAR(255) UNIQUE,
    title VARCHAR(255),
    description TEXT
);

CREATE TABLE IF NOT EXISTS frame_mappings (
    frame_id SERIAL PRIMARY KEY,
    video_id INTEGER REFERENCES videos(id),b
    frame_path VARCHAR(255) UNIQUE,
    pts_time FLOAT,
    frame_idx INT,
    fps INT,
    milvus_id BIGINT
);
"""

cursor.execute(create_tables_sql)
conn.commit()
print("Tables 'videos' and 'frame_mappings' created successfully!")


Tables 'videos' and 'frame_mappings' created successfully!


In [17]:
# check 
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
tables = cursor.fetchall()
print("Tables in database:", tables)


Tables in database: [('videos',), ('frame_mappings',)]


In [ ]:
# Kiểm tra cột của bảng videos
cursor.execute("""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'videos';
""")
columns_videos = cursor.fetchall()
print("Columns in 'videos':")
for col in columns_videos:
    print(col)


Columns in 'videos':
('id', 'integer')
('video_path', 'character varying')
('title', 'character varying')
('description', 'text')


In [ ]:
# Kiểm tra cột của bảng frame_mappings
cursor.execute("""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'frame_mappings';
""")
columns_frames = cursor.fetchall()
print("\nColumns in 'frame_mappings':")
for col in columns_frames:
    print(col)



Columns in 'frame_mappings':
('pts_time', 'double precision')
('video_id', 'integer')
('milvus_id', 'bigint')
('frame_id', 'integer')
('frame_idx', 'integer')
('fps', 'integer')
('frame_path', 'character varying')


In [5]:
# Xóa toàn bộ dữ liệu nhưng giữ cấu trúc bảng
cursor.execute("TRUNCATE TABLE frame_mappings RESTART IDENTITY CASCADE;")
cursor.execute("TRUNCATE TABLE videos RESTART IDENTITY CASCADE;")
conn.commit()

print("✅ Dữ liệu trong 2 bảng 'videos' và 'frame_mappings' đã được xóa.")


✅ Dữ liệu trong 2 bảng 'videos' và 'frame_mappings' đã được xóa.


In [3]:
# kiem tra data
# ================= Kiểm tra bảng videos =================
cursor.execute("SELECT * FROM videos;")
videos_rows = cursor.fetchall()
print("✅ 10 dòng đầu bảng videos:")
for row in videos_rows:
    print(row)

# ================= Kiểm tra bảng frame_mappings =================
cursor.execute("SELECT * FROM frame_mappings LIMIT 10;")
frame_rows = cursor.fetchall()
print("\n✅ 10 dòng đầu bảng frame_mappings:")
for row in frame_rows:
    print(row)


✅ 10 dòng đầu bảng videos:
(1, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V002.mp4', 'L27_V002.mp4', 'Video demo')
(2, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V010.mp4', 'L27_V010.mp4', 'Video demo')
(3, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V007.mp4', 'L27_V007.mp4', 'Video demo')
(4, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V005.mp4', 'L27_V005.mp4', 'Video demo')
(5, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V006.mp4', 'L27_V006.mp4', 'Video demo')
(6, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V001.mp4', 'L27_V001.mp4', 'Video demo')
(7, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V015.mp4', 'L27_V015.mp4', 'Video demo')
(8, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V012.mp4', 'L27_V012.mp4', 'Video demo')
(9, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V004.mp4', 'L27_V004.mp4', 'Video demo')
(10, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V008.mp4', 'L27_V008.mp4', 'Video demo')
(11, '/home/shared/dat